## Experiments with Panel Widgets

In [1]:
import panel as pn

In [2]:
pn.extension()

### Radio buttons select a text entry or a slider

In [3]:
budget_levels = [
    ('$0', 0, 0),
    ('$1M', 1000000, 100000),
    ('$2.5M', 2500000, 250000),
    ('$5M', 5000000, 500000),
    ('$10M', 10000000, 1000000),
    ('$25M', 25000000, 2500000),
    ('$50M', 50000000, 5000000),
    ('$100M', 100000000, 10000000),
]

In [4]:
class BudgetBox(pn.Column):

    def __init__(self):
        super(BudgetBox, self).__init__(margin=(15,0,15,5))
        self.budget_type = pn.widgets.RadioBoxGroup(options=['Return on Investment','Single Budget'], inline=False)
        self.budget_type.param.watch(self.budget_type_cb, ['value'])
        self.levels = [x[0] for x in budget_levels]
        self.map = { x[0]: x[1:] for x in budget_levels }
        self.slider = pn.widgets.DiscreteSlider(
            name='Maximum Budget',
            options = self.levels[:1],
            value = self.levels[0]
        )
        self.slider.param.watch(self.slider_cb, ['value'])
        # self.slider_box = pn.WidgetBox(self.slider)
        self.float_input = pn.widgets.FloatInput(name='Budget', value=0, format='$0,0.00')
        self.row = pn.Row(self.budget_type, self.slider, 'display placeholder')
        self.append(self.row)
        # self.append('display placeholder')

    def budget_type_cb(self, args):
        w = self.float_input if self.budget_type.value.startswith('Single') else self.slider
        self.row[1] = w
        self.row[2] = 'widget changed to ' + self.budget_type.value

    def slider_cb(self, args):
        b, x = self.map[self.slider.value]
        self.row[-1] = f'0 to {b} in steps of {x}'

    def set_budget_max(self, n):
        for i in range(len(budget_levels)-1, -1, -1):
            if n >= budget_levels[i][1]:
                self.slider.options = self.levels[:i+1]
                break


In [5]:
w = BudgetBox()

In [6]:
w

BokehModel(combine_events=True, render_bundle={'docs_json': {'924239e6-3bbd-47c1-acf9-21b99c4dcdf6': {'defs': …

In [7]:
w.set_budget_max(6000000)

### Tabs

In [8]:
class BudgetBox(pn.Column):

    levels = [
        ('$0', 0),
        ('$500K', 500000),
        ('$1M', 1000000),
        ('$2.5M', 2500000),
        ('$5M', 5000000),
        ('$10M', 10000000),
        ('$25M', 25000000),
        ('$50M', 50000000),
        ('$100M', 100000000),
    ]

    boxh = 125
    boxw = 400
    npts = 10

    def __init__(self):
        super(BudgetBox, self).__init__(margin=(15,0,15,5))
        self.labels = [x[0] for x in BudgetBox.levels]
        self.map = { x[0]: x[1] for x in BudgetBox.levels }
        self.tabs = pn.Tabs(
            ('ROI', 
             pn.WidgetBox(
                pn.widgets.DiscreteSlider(
                    options = self.labels[:1], 
                    value = self.labels[0],
                    name = 'Maximum Budget'
                ),
                height=BudgetBox.boxh,
                width=BudgetBox.boxw)),
            ('Fixed', 
             pn.WidgetBox(
                pn.widgets.TextInput(name='Budget Amount', value='$'),
                height=BudgetBox.boxh,
                width=BudgetBox.boxw)),
            ('ℹ️',
            pn.WidgetBox(
                pn.pane.Markdown('''
                    **ROI** (return on investment) will generate a plot that shows the optimal set
                    of gates for several different budget levels.

                    **Fixed** will compute the optimal set of gates for a single budget.
                '''),
                height=BudgetBox.boxh,
                width=BudgetBox.boxw))
        )
        self.append(self.tabs)
        self.slider = self.tabs[0][0]
        self.input = self.tabs[1][0]

    def set_budget_max(self, n):
        for i in range(len(BudgetBox.levels)-1, -1, -1):
            if n >= BudgetBox.levels[i][1]:
                self.slider.options = self.labels[:i+1]
                break

    def values(self):
        if self.tabs.active == 0:
            x = self.map[self.slider.value]
            return x, (x // BudgetBox.npts)
        else:
            s = self.input.value
            if s.startswith('$'):
                s = s[1:]
            return self.input.value, 0


In [9]:
w = BudgetBox()

In [10]:
w

BokehModel(combine_events=True, render_bundle={'docs_json': {'2f427fab-1655-4966-b9cd-a7af70d2c9de': {'defs': …

In [11]:
w.set_budget_max(1000000)

In [12]:
w.values()

(0, 0)

In [13]:
w = pn.widgets.DiscreteSlider(options = ['a','b','c'], value = 'b', name='Letters')

In [14]:
w

BokehModel(combine_events=True, render_bundle={'docs_json': {'fa59f302-4aa1-414b-9f3e-870cc5311148': {'defs': …

In [15]:
w.value

'b'

In [16]:
w.name

'Letters'

In [17]:
from babel.numbers import parse_number, parse_decimal

In [18]:
parse_decimal('12,34.8', locale='en_US')

Decimal('1234.8')

In [19]:
def parse_dollar_amount(s):
    try:
        if s.startswith('$'):
            s = s[1:]
        if s.endswith(('K','M')):
            multiplier = 1000 if s.endswith('K') else 1000000
            res = int(float(s[:-1]) * multiplier)
        elif ',' in s:
            parts = s.split(',')
            assert len(parts[0]) <= 3 and (len(parts) == 1 or all(len(p) == 3 for p in parts[1:]))
            res = int(''.join(parts))
        else:
            res = int(s)
        return res
    except Exception:
        raise ValueError('unexpected format in dollar amount')

In [20]:
parse_dollar_amount('1567')

1567

In [21]:
parse_dollar_amount('$1234')

1234

In [22]:
parse_dollar_amount('$500K')

500000

In [23]:
parse_dollar_amount('$123,456')

123456

In [24]:
parse_dollar_amount('12,345')

12345

In [25]:
parse_dollar_amount('$1.5M')

1500000

In [26]:
parse_dollar_amount('123,45')

ValueError: unexpected format in dollar amount

### Grid

In [28]:
lst = ['alpha', 'beta', 'gamma', 'delta', 'epsilon']

In [35]:
g = pn.GridBox(*lst)

In [38]:
g

BokehModel(combine_events=True, render_bundle={'docs_json': {'4a7b50a7-0697-4426-bc35-a6f9c95d24fa': {'defs': …

In [37]:
g.ncols = 2

In [31]:
nrows = (len(lst) + 1) // 2

In [32]:
for i in range(nrows):
    print(lst[i], lst[i+nrows])

alpha delta
beta epsilon


IndexError: list index out of range